In [0]:
 # Use GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [0]:
# Get google drive authorization
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
# Make google drive directory
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive

In [0]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.getcwd()
os.chdir('/content/drive/My Drive/')
os.getcwd()

'/content/drive/My Drive'

In [0]:
#import library
import pandas as pd
import numpy as np
from glob import glob
import fnmatch
import cv2

import matplotlib.pylab as plt
import itertools
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score

import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D

Using TensorFlow backend.


In [0]:
#create labels
ImagePatch = glob('/content/drive/My Drive/IDC/**/*.png', recursive=True) #replace 'D:/comp5703/input with own input patch
Pattern0 = '*class0.png'
Pattern1 = '*class1.png'
Class0 = fnmatch.filter(ImagePatch, Pattern0)
Class1 = fnmatch.filter(ImagePatch, Pattern1)

def image_proc(lindex,uindex):
    x = []
    y = []

    for img in ImagePatch[lindex:uindex]:
        full_size_image = cv2.imread(img)
        x.append(cv2.resize(full_size_image, (50,50), interpolation=cv2.INTER_CUBIC))
        if img in Class0:
            y.append(0)
        elif img in Class1:
            y.append(1)
        else:
            return
    return x,y

X,Y = image_proc(0,277524)

In [0]:
#data description
original_data = pd.DataFrame()
original_data["image"]=X
original_data["label"]=Y
X1=original_data["image"]
Y1=original_data["label"]

def describe_data(x,y):
    print('Total number: {}'.format(len(x)))
    print('Number of IDC(-): {}'.format(np.sum(y==0)))
    print('Number of IDC(+): {}'.format(np.sum(y==1)))
    print('Percentage of IDC(+) : {:.2f}%'.format(100*np.mean(y)))
describe_data(X1, Y1)

Total number: 277524
Number of IDC(-): 198738
Number of IDC(+): 78786
Percentage of IDC(+) : 28.39%


In [0]:
np.save('X_new.npy',X)
np.save('Y_new.npy',Y)